In [1]:
import pandas as pd
import csv
import pymssql
from ast import literal_eval

In [2]:
def get_unique_strings(list_of_lists):
    unique_strings = set()
    for sublist in list_of_lists:
        for item in sublist:
            if isinstance(item, str):
                unique_strings.add(item)
    return list(unique_strings)

In [3]:
# Connect to MSSQL database
server = 'localhost'
database = 'BockBluster'
username = 'sa'
password = 'thisIsSuperStrong1234'

connection = pymssql.connect(server=server, database=database, user=username, password=password)
cursor = connection.cursor()

# Define the SQL query
query = "SELECT * FROM publisher"

# Execute the query
cursor.execute(query)

# Fetch all the rows
rows = cursor.fetchall()

# Create an empty list
data_list = []

# Iterate over the rows and extract values into the list
for row in rows:
    value1 = row[0]
    value2 = row[1]
    dictionary = {'id': value1, 'name': value2}
    data_list.append(dictionary)

# Close the cursor and the connection
cursor.close()
connection.close()

data_list

[{'id': 1, 'name': 'Perfect World Pictures'},
 {'id': 2, 'name': 'Zide-Perry Productions'},
 {'id': 3, 'name': 'Hehe (Shanghai) Pictures'},
 {'id': 4, 'name': 'Robert Simonds Productions'},
 {'id': 5, 'name': 'Village Roadshow Pictures'},
 {'id': 6, 'name': 'Sprockets Music'},
 {'id': 7, 'name': 'EMI Films'},
 {'id': 8, 'name': 'Playtone'},
 {'id': 9, 'name': 'Red Hour Films'},
 {'id': 10, 'name': 'TriStar Pictures'},
 {'id': 11, 'name': 'Allison Shearmur Productions'},
 {'id': 12, 'name': 'Goldmann Pictures'},
 {'id': 13, 'name': 'Las Producciones del Escorpión'},
 {'id': 14, 'name': 'Annapurna Pictures'},
 {'id': 15, 'name': 'Tom DeSanto/Don Murphy Production'},
 {'id': 16, 'name': 'Jetsen Cultural Industry Group'},
 {'id': 17, 'name': 'The Kennedy/Marshall Company'},
 {'id': 18, 'name': 'Pacific Western'},
 {'id': 19, 'name': 'Prime Focus'},
 {'id': 20, 'name': 'Perfect Storm Entertainment'},
 {'id': 21, 'name': 'Great Oaks Entertainment'},
 {'id': 22, 'name': 'Thunder Road Pictures

In [4]:
df = pd.read_csv('../data/movies_new.csv')

In [5]:
df['publishers'] = df['publishers'].apply(literal_eval)

df['publishers'] = df['publishers'].apply(lambda x: list(set(x)))

df_new = df[['id', 'publishers']]

df_new = df_new.explode('publishers')

df_new

,id,publishers
0,tt0499549,Twentieth Century Fox
0,tt0499549,Dune Entertainment
0,tt0499549,Lightstorm Entertainment
1,tt4154796,Marvel Studios
1,tt4154796,Walt Disney Pictures
...,...,...
998,tt0299977,China Film Co-Production Corporation
998,tt0299977,Zhang Yimou Studio
998,tt0299977,Edko Films
999,tt0405422,Universal Pictures


In [6]:
movies_list = df_new.to_dict(orient='records')


mapping = []
for movie in movies_list:
    movie_id = movie['id']
    publisher_name = movie['publishers']
    for publisher in data_list:
        if publisher['name'] == publisher_name:
            publisher_id = publisher['id']
            new_movie = {'movie_id': movie_id, 'publisher_id': publisher_id}
            mapping.append(new_movie)
            break

result = [d for d in mapping if d['movie_id'] == 'tt7286456']

mapping

[{'movie_id': 'tt0499549', 'publisher_id': 518},
 {'movie_id': 'tt0499549', 'publisher_id': 357},
 {'movie_id': 'tt0499549', 'publisher_id': 213},
 {'movie_id': 'tt4154796', 'publisher_id': 52},
 {'movie_id': 'tt4154796', 'publisher_id': 515},
 {'movie_id': 'tt0120338', 'publisher_id': 518},
 {'movie_id': 'tt0120338', 'publisher_id': 213},
 {'movie_id': 'tt0120338', 'publisher_id': 506},
 {'movie_id': 'tt2488496', 'publisher_id': 37},
 {'movie_id': 'tt2488496', 'publisher_id': 194},
 {'movie_id': 'tt2488496', 'publisher_id': 515},
 {'movie_id': 'tt4154756', 'publisher_id': 52},
 {'movie_id': 'tt4154756', 'publisher_id': 244},
 {'movie_id': 'tt4154756', 'publisher_id': 137},
 {'movie_id': 'tt10872600', 'publisher_id': 293},
 {'movie_id': 'tt10872600', 'publisher_id': 387},
 {'movie_id': 'tt10872600', 'publisher_id': 52},
 {'movie_id': 'tt0369610', 'publisher_id': 335},
 {'movie_id': 'tt0369610', 'publisher_id': 567},
 {'movie_id': 'tt0369610', 'publisher_id': 426},
 {'movie_id': 'tt6105

In [7]:
# Connect to MSSQL database
server = 'localhost'
database = 'BockBluster'
username = 'sa'
password = 'thisIsSuperStrong1234'

connection = pymssql.connect(server=server, database=database, user=username, password=password)
cursor = connection.cursor()

# Insert dictionaries into the table
cursor = connection.cursor()
for dictionary in mapping:
    cursor.execute("INSERT INTO movie_publishers (movie_id, publisher_id) VALUES (%s, %s)", (dictionary['movie_id'], dictionary['publisher_id']))

# Commit the changes and close the connection
connection.commit()
connection.close()